# Serializer showdown

A simple benchmark on the performance of various serializers.

As with all benchmarks, it is a decent indictator for your use case, but not a definite one. Benchmark your use case (fork this repo if you want!) and verify yourself before acting on these results.

In [2]:
import attr
import time
import pprint
import pydantic
from schematics.models import Model
from schematics.types import StringType, IntType
from marshmallow import Schema, fields
from tabulate import tabulate
from pydantic import BaseModel
from cattr import structure, unstructure



@attr.s
class PersonAttrs:
    name = attr.ib(validator=[
        attr.validators.instance_of(str)
    ])
    age = attr.ib(validator=[
        attr.validators.instance_of(int)
    ])
    
class PersonClass:
    def __init__(self, name, age):
        if not isinstance(name, str):
            raise ValueError("name is not a string")
        if not isinstance(age, int):
            raise ValueError("age is not an integer")
        self.name = name
        self.age = age    
        
        
class PersonClassRaw:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        

class PersonSchematics(Model):
    name = StringType()
    age = IntType()


class PersonMarshmallow(Schema):
    name = fields.Str()
    age = fields.Int()

    
class PersonPydantic(BaseModel):
    name: str = ...
    age: int = ...

marshmallow_schema = PersonMarshmallow()

target = {"name": "Mario Luigi", "age": 32}

TRIALS = 10000


def benchmark(name, func):
    start_time = time.time()
    for i in range(TRIALS):
        func()
    elapsed_time = time.time() - start_time
    return [name, elapsed_time, TRIALS]

data = [["library", "execution_time (seconds)", "iterations"]]
data.append(benchmark("schematics", lambda: PersonSchematics(target).validate()))
data.append(benchmark("marshmallow", lambda: marshmallow_schema.dump(target)))
data.append(benchmark("attrs", lambda: PersonAttrs(**target)))
data.append(benchmark("pydantic", lambda: PersonPydantic(**target)))
data.append(benchmark("class", lambda: PersonClass(**target)))
data.append(benchmark("class_no_validation", lambda: PersonClassRaw(**target)))
data.append(benchmark("cattrs", lambda: structure(target, PersonAttrs)))


print(tabulate(data, headers="firstrow"))

library                execution_time (seconds)    iterations
-------------------  --------------------------  ------------
schematics                           1.67589            10000
marshmallow                          0.322552           10000
attrs                                0.0243888          10000
pydantic                             0.0982409          10000
class                                0.00822067         10000
class_no_validation                  0.00591969         10000
cattrs                               0.0697694          10000
